In [16]:
# 1. Imports
import pandas as pd
from pathlib import Path
import sys
import os

import importlib
 

 

In [ ]:

import requests
import pandas as pd
from datetime import datetime, timedelta
import json

 

API_KEY = "cc099cfca8464b55b73357eec92da761"

def run_query(query):
  try:
      response = requests.post(
          url="https://api.trafikinfo.trafikverket.se/v2/data.json",
          data=query.encode("utf-8"),
          headers={"Content-Type": "text/xml"},
      )

      # Check for HTTP errors
      response.raise_for_status()

      # Try parsing JSON
      try:
          data = response.json()
      except json.JSONDecodeError as e:
          print("❌ Failed to parse JSON response:")
          print(response.text)
          raise e

      # Check for API-level errors in the response
      if "RESPONSE" in data and "RESULT" in data["RESPONSE"]:
          result = data["RESPONSE"]["RESULT"][0]
          if "ERROR" in result:
              print("⚠️ API Error:")
              print(json.dumps(result["ERROR"], indent=2))
          else:
              print("✅ Success")
      else:
          print("⚠️ Unexpected response format:")
          print(json.dumps(data, indent=2))

  except requests.exceptions.RequestException as e:
      print("🚨 HTTP Request failed:")
      print(e)
  return response

def fetch_stations():
    query = f"""
    <REQUEST>
        <LOGIN authenticationkey='{API_KEY}' />
          <QUERY  objecttype="TrainStation" namespace="rail.infrastructure" schemaversion="1.5">
            <FILTER>
    
            </FILTER>
            <EXCLUDE>LocationInformationText</EXCLUDE>
            <EXCLUDE>ModifiedTime</EXCLUDE>
            <EXCLUDE>PlatformLine</EXCLUDE>
            
          </QUERY>
    </REQUEST>
    """

    print(query)
    response = run_query(query)


    data = response.json()


    data = response.json()["RESPONSE"]["RESULT"][0]["TrainStation"]
    df = pd.json_normalize(data)


    return df

def fetch_departures(station_code="Cst"):
    now = datetime.now()
    now_str = now.strftime("%Y-%m-%dT%H:%M:%S")
    one_hour_later = (now + timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S")

    print (now_str)

    from_time = '2025-04-01T00:00:00'
                  
    to_time = '2025-05-01T00:00:00'

    query = f"""
    <REQUEST>
      <LOGIN authenticationkey='{API_KEY}' />
      <QUERY objecttype='TrainAnnouncement' orderby='AdvertisedTimeAtLocation' schemaversion="1.9" >  
        <FILTER>
          <AND>
            <EQ name='Advertised' value='true'  />

            
            
            
            <EQ name='ActivityType' value='Avgang' />
          </AND>
        </FILTER>
        <EXCLUDE>ActivityId</EXCLUDE>
        <EXCLUDE>WebLink</EXCLUDE>
        <EXCLUDE>WebLinkName</EXCLUDE>
        <EXCLUDE>OperationalTransportIdentifiers</EXCLUDE>
        <EXCLUDE>OtherInformation</EXCLUDE>
        <EXCLUDE>ViaToLocation</EXCLUDE>
        <EXCLUDE>MobileWebLink</EXCLUDE>
        <EXCLUDE>TimeAtLocationWithSeconds</EXCLUDE>
        <EXCLUDE>TypeOfTraffic</EXCLUDE>
        <EXCLUDE>ProductInformation</EXCLUDE>

        


      </QUERY>
    </REQUEST>
    """

    print(query)
    response = run_query(query)


    data = response.json()


    data = response.json()["RESPONSE"]["RESULT"][0]["TrainAnnouncement"]
    df = pd.json_normalize(data)


    return df



In [ ]:
 

df_raw = fetch_departures()

import pandas as pd

df = pd.DataFrame(df_raw)
print(df.head())


# Create data/raw folder if needed
Path("../data/raw").mkdir(parents=True, exist_ok=True)

# Save CSV
df.to_csv("../data/raw/train_departures.csv", index=False)

In [22]:
 


df_raw = fetch_stations()

import pandas as pd

df = pd.DataFrame(df_raw)
print(df.head())


# Create data/raw folder if needed
Path("../data/raw").mkdir(parents=True, exist_ok=True)

# Save CSV
df.to_csv("../data/raw/stations.csv", index=False)


    <REQUEST>
        <LOGIN authenticationkey='cc099cfca8464b55b73357eec92da761' />
          <QUERY  objecttype="TrainStation" namespace="rail.infrastructure" schemaversion="1.5">
            <FILTER>
    
            </FILTER>
            <EXCLUDE>LocationInformationText</EXCLUDE>
            <EXCLUDE>ModifiedTime</EXCLUDE>
            
          </QUERY>
    </REQUEST>
    
✅ Success
   Advertised AdvertisedLocationName AdvertisedShortLocationName  \
0        True               Alingsås                    Alingsås   
1        True               Anneberg                    Anneberg   
2        True           Abisko Östra                    Abisko Ö   
3        True                  Aneby                       Aneby   
4        True                  Aspen                       Aspen   

  PrimaryLocationCode CountryCode CountyNo  Deleted LocationSignature  \
0                 119          SE     [14]    False                 A   
1                 155          SE     [13]    False  